In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

np.random.seed(503)

In [2]:
# General parameters for the script
target_name = 'RegisteredInTargetPeriod'  # Target variable
features = [
    'DaysSinceLastRegistration', 'DaysSinceFirstRegistration',
    'PastRegistrations', 'LastDonationLocation_Center',
    'LastDonationType_Platelets', 'CenterRegistrationProportion', 'DonationsPerDay',
    'PlateletRegistrationProportion'
]

# scoring = 'precision'  # Precision: TP / (TP + FN), or % of predicted positives that are actually positive
# scoring = 'balanced_accuracy'
# scoring = 'roc_auc'
scoring = 'average_precision'  # Summarizes a precision-recall curve: weighted mean of precisions achieved at each threshold for binary classification, appropriate for imbalanced data

# Decide whether we're loading a subset or the full set
# dataset_size = 'partial'
dataset_size = 'full'

if dataset_size == 'full':
    file_names = {
        'X': 'X_train_full.csv',
        'y': 'y_train_full.csv'
    }
elif dataset_size == 'partial':
    file_names = {
        'X': 'X_train.csv',
        'y': 'y_train.csv'
    }

In [3]:
# Load data
with open('../../data/processed/dtypes.json') as in_file:
    non_date_dtypes = json.load(in_file)

with open('../../data/processed/date_types.json') as in_file:
    date_dtypes = json.load(in_file)

date_cols = list(date_dtypes)

# Read data, specifically parsing date columns as dates and only picking the features + target
X_train = pd.read_csv('../../data/processed/{0}'.format(file_names['X']), dtype=non_date_dtypes, index_col=0)
y_train = pd.read_csv('../../data/processed/{0}'.format(file_names['y']), index_col=0)

In [4]:
X_train.head()

,DaysSinceLastRegistration,DaysSinceFirstRegistration,PastRegistrations,LastDonationLocation_Center,LastDonationType_Platelets,CenterRegistrationProportion,DonationsPerDay,PlateletRegistrationProportion
2041464,255,255,1,0.0,0.0,0.000000,0.003922,0.0
10182607,581,581,1,0.0,0.0,0.000000,0.001721,0.0
3223306,112,722,2,0.0,0.0,0.000000,0.002770,0.0
9056300,28,487,3,0.0,0.0,0.333333,0.006160,0.0
3910737,23,561,5,1.0,0.0,0.200000,0.008913,0.0


In [5]:
y_train.head()

,RegisteredInTargetPeriod
2041464,0
10182607,0
3223306,1
9056300,1
3910737,0


In [6]:
# This code from: http://www.davidsbatista.net/blog/2018/02/23/model_optimization/

class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=3, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

In [7]:
models = {
    'LogisticRegression': LogisticRegression(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'GaussianNB': GaussianNB()
}

params = {
    'LogisticRegression': { 'penalty': ['none', 'l2'] },
    'RandomForestClassifier': { 'n_estimators': [64] },
    'AdaBoostClassifier': { 'n_estimators': [64] },
    'GradientBoostingClassifier': { 'n_estimators': [128], 'learning_rate': [0.5] },
    'KNeighborsClassifier': { 'n_neighbors': [5, 10] },
    'DecisionTreeClassifier': {},
    'GaussianNB': {}
}

In [8]:
# Run grid search CV across all estimators
helper = EstimatorSelectionHelper(models, params)
helper.fit(X_train, y_train, scoring=scoring, n_jobs=8)

Running GridSearchCV for LogisticRegression.
Fitting 3 folds for each of 2 candidates, totalling 6 fits
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of   6 | elapsed:  1.3min finished
Running GridSearchCV for RandomForestClassifier.
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   3 | elapsed: 25.7min finished
Running GridSearchCV for AdaBoostClassifier.
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   3 | elapsed: 12.0min finished
Running GridSearchCV for GradientBoostingClassifier.
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   3 | elapsed: 18.5min f

In [9]:
helper.score_summary(sort_by='mean_score')

LogisticRegression
RandomForestClassifier
AdaBoostClassifier
GradientBoostingClassifier
KNeighborsClassifier
DecisionTreeClassifier
GaussianNB


,estimator,min_score,mean_score,max_score,std_score,penalty,n_estimators,learning_rate,n_neighbors
4,GradientBoostingClassifier,0.472926,0.473955,0.474727,0.000757219,NaN,128,0.5,NaN
3,AdaBoostClassifier,0.45347,0.454745,0.45541,0.000901456,NaN,64,NaN,NaN
1,LogisticRegression,0.432962,0.434453,0.435553,0.00109321,l2,NaN,NaN,NaN
0,LogisticRegression,0.432962,0.434208,0.434843,0.000880985,none,NaN,NaN,NaN
2,RandomForestClassifier,0.392597,0.393142,0.393516,0.00039441,NaN,64,NaN,NaN
6,KNeighborsClassifier,0.377944,0.37847,0.379522,0.000743605,NaN,NaN,NaN,10
8,GaussianNB,0.345631,0.345931,0.346116,0.00021423,NaN,NaN,NaN,NaN
5,KNeighborsClassifier,0.325234,0.32675,0.328592,0.00138991,NaN,NaN,NaN,5
7,DecisionTreeClassifier,0.2957,0.296026,0.29662,0.000420343,NaN,NaN,NaN,NaN


# Results

## Run 1
* Partial dataset: `GradientBoostingClassifier` with `n_estimators` = 128, `learning_rate` = 0.5 had the best performance with a mean average precision of **0.674885**.
* Full dataset: `RandomForestClassifier` with `n_estimators` = 64 had the best performance with a mean average precision of **0.799422**.

## Run 2
* Full dataset: `GradientBoostingClassifier` with `n_estimators` = 128, `learning_rate` = 0.5, `mean_score` = 0.703001

